In [2]:
// setup
#!import config/Settings.cs
#r "nuget: Microsoft.SemanticKernel, 1.1.0"

using Microsoft.SemanticKernel;

//Create Kernel builder
var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
// Configure AI service credentials used by the kernel
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.1.0

# Plugins

Mit Plugins erweitern wir Semantic Kernel um die möglichkeit GPT als SDK für unsere Anwendung zu nutzen.

Plugins beinhalten eine oder mehrere Funktionen um bestimmte Aufgaben zu erledigen. Als Teil unserer Pipeline, können wir dann Plugins "aneinander reihen" um unsere gesamt Lösung zu schaffen.

## Funktionen

Semantic Kernel unterscheiden zwischen 2 verschiedenen Arten von Funktionen.
* Semantic Functions
* Native Functions

## Semantic Functions
Semantic Functions sind Funktionen die an GPT geschickt werden um z.B. Text zu analysieren oder zu extrahieren.
Semantic Functions bestehen aus 2 Komponenten, einer Config und einem Prompt

## Plugins 
Plugins können anhand von OpenApi Specs, Ordnern mit semantischen Funktionen oder für native Funktionen direkt im Code importiert werden

**SKPrompt**

Wie der Name schon suggeriert ist diese Komponente nichts weiter als ein Prompt

In [3]:
var prompt = $$$"""
                Du bist eine KI die den Usern mit ihren Anfragen hilft.
                Du sprichst lediglich Deutsch und Esperanto.
                Wenn du mal nicht weiter weißt oder dir das wissen fehlt eine frage zu beantworten, dann meckere über den Kapitalismus und wie er die menschen verdirbt.
                
                {{$input}}
                """;

**Config**

Die Config beschreibt die Funktion und Parameter einer Semantic Function

In [4]:
var promptConfig = new PromptTemplateConfig
{
    InputVariables = {
        new Microsoft.SemanticKernel.InputVariable {
                Name = "input",
                Description = "Anliegen des Nutzers",
                Default = ""
            }
    },
    Description = "Hilft nutzer bei deren Anliegen",
};

**Semantic Functions serialisieren**

Semantic Functions müssen aber nicht im Code geschrieben werden, wir können sie serialisieren und in dateien speichern.

Die Dateistruktur dabei sieht folgender maßen aus:

```text
-- PluginName
  -- FunktionName
    -- config.json
    -- skprompt.txt
```

Unsere Config von oben sieht dann so aus:
**config.json**
```json
{
  "description": "Hilft nutzer bei deren Anliegen",
  "input_variables": [
    {
      "name": "input",
      "description": "Joke subject",
      "default": ""
    },
    {
      "name": "style",
      "description": "Give a hint about the desired joke style",
      "default": ""
    }
  ]
}
```


Hier ist ein Beispiel für eine inline Semantic Function, welche einen TLDR for einen Wiki-Text erstellt.




In [10]:
string textToSummarize = """
                   Microsoft Build (often stylised as //build/) is an annual conference event held by Microsoft,
                   aimed at software engineers and web developers using Windows, Microsoft Azure and other Microsoft technologies.
                   First held in 2011, it serves as a successor for Microsoft's previous developer events,
                   the Professional Developers Conference (an infrequent event which covered development of software for the Windows operating system)
                   and MIX (which covered web development centering on Microsoft technology such as Silverlight and ASP.net).
                   The attendee price was (US)$2,195 in 2016, up from $2,095 in 2015. It sold out quickly,
                   within one minute of the registration site opening in 2016.

                   The event has been held at a large convention center, or purpose-built meeting space on the Microsoft Campus.
                   The Keynote on the first day has been led by the Microsoft CEO addressing the press and developers.
                   It has been the place to announce the general technology milestones for developers.
                   There are breakout sessions conducted by engineers and program managers, most often Microsoft employees representing their particular initiatives.
                   The keynote on the second day often includes deeper dives into technology. Thousands of developers and technologists from all over the world attend.

                   The 2023 conference was announced on March 14, 2023. A free online part was held from May 23 to 24,
                   and an in-person part was held in Seattle from May 23 to 25 with workshops on May 22. Tickets for the in-person event are $1525 and workshops are $225.

                   Microsoft Build 2023 focused heavily on artificial intelligence and its integration across Microsoft's products and services.

                   Microsoft introduced its AI chatbot, Copilot, to Windows 11. Copilot is already integrated into Edge, Office apps, and GitHub.
                   It will be available from June, offering users a range of assistance, from simple to complex tasks.
                   Microsoft announced the integration of Bing with OpenAI's ChatGPT, making Bing the default search engine for ChatGPT.
                   This functionality is first available for ChatGPT Plus users and will be enabled for all free ChatGPT users later.
                   In addition to AI-focused announcements, Microsoft introduced updates to Windows 11, Microsoft 365, Edge, and Teams.
                   The company also expanded its plugin platform in collaboration with OpenAI, allowing developers to submit their AI experiences to the Microsoft Store on Windows
                   """;

In [11]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var function = kernel.CreateFunctionFromPrompt(skPrompt);

var summary = await function.InvokeAsync(kernel, new() { ["input"] = textToSummarize });

Console.WriteLine(summary);


Microsoft Build 2023: AI emphasis.
